In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pickle
from mutation import *
from crossover import *
from rmp import *
from mfea import *
from task import *
from llm import *
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os

In [21]:
# load_dotenv()

# GPT_API_KEY = os.getenv("GPT_API_KEY")

# llm = GPTModel(GPT_API_KEY, "gpt-3.5-turbo-0125", 0.7)

In [5]:
cec2017_tasks = get_10_tasks()

In [6]:
num_indis_per_task = 100
indi_len = 50
mutation = PolynomialMutation(5, 0.02)
crossover = SBXCrossover(mutation, eta=2)
# crossover = BLXCrossover()
rmp = AdaptiveRMPMatrix(rmp_pop_size=5, num_gen=0, pc=0.8, pm=0.1)

np.random.seed(0)
amfea = AMFEA(num_indis_per_task, indi_len, cec2017_tasks, crossover, mutation, rmp)

Initialization:
Task 0:
Best Fitness: 125473.96699307997
Mean Fitness: 166392.121167927

Task 1:
Best Fitness: 319204.0909081603
Mean Fitness: 503987.30720511527

Task 2:
Best Fitness: 346653.31463481847
Mean Fitness: 486737.256905658

Task 3:
Best Fitness: 41.06041433751163
Mean Fitness: 49.71421741800349

Task 4:
Best Fitness: 4058761661.357497
Mean Fitness: 6371677088.876898

Task 5:
Best Fitness: 21.482750101043788
Mean Fitness: 21.715487176250566

Task 6:
Best Fitness: 86.24596181476284
Mean Fitness: 99.85596727380062

Task 7:
Best Fitness: 18243.689210172182
Mean Fitness: 21153.768897608556

Task 8:
Best Fitness: 74.8950497645563
Mean Fitness: 122.64795239117853

Task 9:
Best Fitness: 88658.75615256254
Mean Fitness: 124389.07711105635



In [7]:
bfs, mfs = amfea.fit(num_gen=1000,monitor=True, monitor_rate=100, llm_rate=100)

Gen 0
Evaluation count: 0
Task 0, Best: 82192.89040593142, Avg: 138839.9283654692
Task 1, Best: 316014.31410111557, Avg: 430087.0594044205
Task 2, Best: 292894.69300041284, Avg: 400968.00669433194
Task 3, Best: 41.06041433751163, Avg: 45.9374969089205
Task 4, Best: 2027943133.4205256, Avg: 4741543962.93781
Task 5, Best: 21.33689326217712, Avg: 21.585582604853993
Task 6, Best: 80.48434729630105, Avg: 92.56483821832202
Task 7, Best: 17272.06912677073, Avg: 19688.46005266694
Task 8, Best: 73.94610143210085, Avg: 101.46741321674455
Task 9, Best: 75247.30020949453, Avg: 105568.75893567922
Time taken each gen: 0.1871 seconds

Creating strategy
Evaluating strategy
Strategy:
 Allocate higher mating probabilities to tasks with higher genetic diversity
Adjust mating probabilities based on convergence rates to promote exploration or exploitation
Increase mating probabilities for tasks with high fitness variance to encourage genetic diversity
Prioritize mating probabilities for tasks with higher o

In [25]:
# num_tasks = len(tasks)
# fig, axes = plt.subplots(num_tasks, 2)
# fig.tight_layout()
# for i in range(num_tasks):
#     axes[i][0].plot(bfs[i])
#     axes[i][1].plot(mfs[i])